# **Baselines**

In [ ]:
#### Load the required packages
import os
import re

In [ ]:
#### Point it to the directory where your Test Data is present
input_dir = "/content/drive/MyDrive/sub3_ph22/"

In [ ]:
#### List all the folders present in the directory
list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]

In [ ]:
#### Load the Test Data

input_stanza_list = [] #Stores individual lines from Stanza_out.txt file
input_stanza_len = [] #Stores the number of lines present in each Stanza_out.txt file 
input_sent_num_list = [] #Stores individual lines from sentences.txt file
file_name_list = [] #Stores the name of individual files
input_entity_list = [] #Stores the list of all phrases from entities.txt


for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    for files in os.listdir(input_dir + fls + '/' + i + "/"):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x,stanza_lines.lower().splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)  
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
      if files.endswith("entities.txt"):
        entities_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,(entities_file.read()).splitlines())) # filter empty strings and split into lines
        input_entity_list.append(entities_list)
    count=count+1
    temp_list_code_triple = []
    
    file_name_list.append(fls + '/' + str(i))
  print("completed",fls, count)

In [ ]:
print(len(file_name_list))
print()
print(file_name_list[1])
print()
print(len(input_stanza_list))
print()
print(input_stanza_list[1])
print()
print(len(input_entity_list))
print()
print(input_entity_list[1])

In [ ]:
#### Checks if that sentence has a punctuation or not.
#### We extract out headings from stanza_out.txt file by checking if that sentence contains a punctuation or not.
import string
result = string.punctuation
def check_punctuation(sentence):
  flag = 0
  for i in sentence: 
    if i in result:   
        flag = 1
        break
  if flag == 1:     
    return 1
  else:
    return 0         

In [ ]:
#### This block finds out all the files and the corresponding sentence numbers which should belong to Baselines Information unit
#### We store all the sentence numbers between a "Baseline" heading and the next consequetive heading (basically all sentences in the "baseline" paragraph)

regex = r"comp" #Most of the baseline heading have "comp" as starting word. Eg. "Comparision", "Compare", "Competitive", etc.
regex4  = r"baseline" #Most of the headings also have "baseline" word in them
regex5 = r"compr" #We do not want words such as "compressed", "compromised", etc.
regex6 = r"compu" #We do not want words such as "computed", "computes", etc.
regex7 = r"compl" #We do not want words such as "complain", "comply", etc.
regex8 = r"compo" #We do not want words such as "composed", "composition", etc.
c3 = 0 #Keeps a count of all baseline headings found

dict_of_baselines = {} #Stores the filename as key and the eligible baseline sentence numbers as values 

for i in zip(input_stanza_list, file_name_list):
  print(i[1]) #Prints file name
  found1 = 0
  for counterr, j in enumerate(i[0]):
    flag = check_punctuation(j) #This line checks if that sentence is a heading or not by using the punctuation heurisitic 
    if (flag==0): 
      if (found1): #found1 will be 1 when we had previously found a baseline heading. So in this block, we add all the sentences between the two headings(i.e. the baseline heading and the next heading found)
        if (i[1] not in dict_of_baselines.keys()):
          dict_of_baselines[i[1]] = [] #Make a new empty list in that key if it was not already present
        dict_of_baselines[i[1]] = dict_of_baselines[i[1]] + list(range(found1+1, counterr+1)) #Add all the sentence numbers between the previous baseline heading and the current found heading
        found1 = 0 #Again put it to its default value

      split_list = j.split() 
      search_for_comp = re.search(regex, j)
      search_for_baseline = re.search(regex4, j)
      search_for_compr = re.search(regex5, j)
      search_for_compu = re.search(regex6, j)
      search_for_compl = re.search(regex7, j)
      search_for_compo = re.search(regex8, j)
      #The Heading is a Baseline Heading if it contains either "comp" or "baseline" and is not something having "compr", "compu", "compl", "compo".
      if ((search_for_baseline) or (search_for_comp)) and (search_for_compr==None) and (search_for_compu==None) and (search_for_compl==None) and (search_for_compo==None):
        print("This is a baseline heading")
        print(j)
        c3 += 1
        found1 = counterr #Store the sentence number of that Heading

          
print(c3)   

In [ ]:
#### For reference
print(len(dict_of_baselines))
print()
print(dict_of_baselines["natural_language_inference/1"])
print()
print(dict_of_baselines["face_alignment/4"])
print()
print(dict_of_baselines["document_classification/18"])

In [ ]:
#### Out of the selected baseline sentences, only those sentences are used which are contribution sentences(i.e. in sentences.txt file)
dict_of_selected_lines_for_baseline = {}
for i in zip(file_name_list, input_sent_num_list):
  if (i[0] in dict_of_baselines.keys()):
    dict_of_selected_lines_for_baseline[i[0]] = []
    for j in i[1]: 
      if ((int)(j) in dict_of_baselines[i[0]]): #Main checking condition where the baseline sentence is included only if it is a contribution sentence
        dict_of_selected_lines_for_baseline[i[0]] = dict_of_selected_lines_for_baseline[i[0]] + [(int)(j)]

In [ ]:
#### For reference
print(len(dict_of_selected_lines_for_baseline))
print()
print(dict_of_selected_lines_for_baseline["natural_language_inference/1"])
print()
print(dict_of_selected_lines_for_baseline["document_classification/18"])

In [ ]:
##### Phrases arrangement in ascending order according to their sentence numbers and their starting character numbers 
import copy
train_final_phrases_list = []
for i in range(len(input_stanza_list)):
  entity_list = [j.split('\t') for j in input_entity_list[i]]
  entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) 
  train_final_phrases_list = train_final_phrases_list + [entity_list]

In [ ]:
#### For reference
print(len(train_final_phrases_list))
print()
print(train_final_phrases_list[3])

In [ ]:
#### Here we form the triples using the rule that all three consequetive triples in the same sentence are treated as a triple.
baseline_predicted_triples = [] #Stores the triples formed for each file 

for j, name_of_file in enumerate(file_name_list):
  temp_store_list = []
  if name_of_file in dict_of_selected_lines_for_baseline.keys(): 
    for i in range(len(train_final_phrases_list[j])):
      if (int)(train_final_phrases_list[j][i][0]) in dict_of_selected_lines_for_baseline[name_of_file]: #Form triples only for those files in which "baselines" sentences are found
        try:
          #Check if the sentence number is same for the three consequetive phrases
          if((train_final_phrases_list[j][i][0]==train_final_phrases_list[j][i+1][0])  and (train_final_phrases_list[j][i][0]==train_final_phrases_list[j][i+2][0])):
            temp_store_list = temp_store_list + [[train_final_phrases_list[j][i][3], train_final_phrases_list[j][i+1][3], train_final_phrases_list[j][i+2][3]]]
        except Exception as e:
          continue
  baseline_predicted_triples = baseline_predicted_triples + [temp_store_list]

In [ ]:
#### For reference
print(len(baseline_predicted_triples))
print()
print([j for j in baseline_predicted_triples if len(j)!=0])

In [ ]:
#### Write the formed triples only in those files which have Baselines information unit already predicted in them
#### Note that we are not predicting Baselines Information unit.
#### We are only forming triples for the files which have Baseline Information Unit already predicted in them (from the Information unit prediction step). 
for i in zip(file_name_list, baseline_predicted_triples):
  #Check whether we have formed baseline triples for this file AND whether this file already contains Baseline Information Unit.
  if i[0] in dict_of_selected_lines_for_baseline.keys() and ("baselines.txt" in os.listdir(input_dir+i[0]+"/"+"triples/")): 
    output_file = open(input_dir + i[0] + "/" + "triples/" + "baselines.txt","w")
    output_file.write("(Contribution||has||Baselines)" + "\n")
    for j in i[1]:
      output_file.write("(" + j[0] + "||" + j[1] + "||" + j[2]  + ")" + "\n")     
    output_file.close()





# **Ablation Analysis**

In [ ]:
#### Load the required packages
import os
import re

In [ ]:
#### Point it to the directory where your Test Data is present
input_dir = "/content/drive/MyDrive/sub3_ph22/"

In [ ]:
#### List all the folders present in the directory
list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]

In [ ]:
#### Load the Test Data

input_stanza_list = [] #Stores individual lines from Stanza_out.txt file
input_stanza_len = [] #Stores the number of lines present in each Stanza_out.txt file 
input_sent_num_list = [] #Stores individual lines from sentences.txt file
file_name_list = [] #Stores the name of individual files
input_entity_list = [] #Stores the list of all phrases from entities.txt


for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    for files in os.listdir(input_dir + fls + '/' + i + "/"):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read()
        stanza_lines_list = list(filter(None,map(lambda x:x,stanza_lines.lower().splitlines()))) # filter empty strings and split into lines
        input_stanza_len.append(len(stanza_lines_list))
        input_stanza_list.append(stanza_lines_list)  
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,sentence_file.read().splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(sentence_num_list)
      if files.endswith("entities.txt"):
        entities_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,(entities_file.read()).splitlines())) # filter empty strings and split into lines
        input_entity_list.append(entities_list)
    count=count+1
    temp_list_code_triple = []
    
    file_name_list.append(fls + '/' + str(i))
  print("completed",fls, count)

In [ ]:
print(len(file_name_list))
print()
print(file_name_list[1])
print()
print(len(input_stanza_list))
print()
print(input_stanza_list[1])
print()
print(len(input_entity_list))
print()
print(input_entity_list[1])

In [ ]:
#### Checks if that sentence has a punctuation or not.
#### We extract out headings from stanza_out.txt file by checking if that sentence contains a punctuation or not.
import string
result = string.punctuation
def check_punctuation(sentence):
  flag = 0
  for i in sentence: 
    if i in result:   
        flag = 1
        break
  if flag == 1:     
    return 1
  else:
    return 0         

In [ ]:
#### This block finds out all the files and the corresponding sentence numbers which should belong to Ablation Analysis Information unit
#### We store all the sentence numbers between an "Ablation Analysis" heading and the next consequetive heading (basically all sentences in the "Ablation Analysis" paragraph)

regex = r"ablation" #Most of the Ablation Analysis headings have "ablation" as starting word.
c3 = 0 #Keeps a count of all Ablation Analysis headings found

dict_of_baselines = {} #Stores the filename as key and the eligible Ablation Analysis sentence numbers as values 

for i in zip(input_stanza_list, file_name_list):
  print(i[1]) #Prints file name
  found1 = 0
  for counterr, j in enumerate(i[0]):
    flag = check_punctuation(j) #This line checks if that sentence is a heading or not by using the punctuation heurisitic. 
    if (flag==0): 
      if (found1): #found1 will be 1 when we had previously found an Ablation Analysis heading. So in this block, we add all the sentences between the two headings(i.e. the Ablation Analysis heading and the next heading found)
        if (i[1] not in dict_of_baselines.keys()):
          dict_of_baselines[i[1]] = [] #Make a new empty list in that key if it was not already present
        dict_of_baselines[i[1]] = dict_of_baselines[i[1]] + list(range(found1+1, counterr+1)) #Add all the sentence numbers between the previous Ablation Analysis heading and the current found heading
        found1 = 0 #Again put it to its default value

      split_list = j.split() 
      search_for_ablation = re.search(regex, j)
      
      #The Heading is an Ablation Analysis Heading if it contains "ablation" word in it.
      if (search_for_ablation):
        print("This is an Ablation Analysis heading")
        print(j)
        c3 += 1
        found1 = counterr #Store the sentence number of that Heading

          
print(c3)   

In [ ]:
#### Out of the selected Ablation Analysis sentences, only those sentences are used which are contribution sentences(i.e. in sentences.txt file)
dict_of_selected_lines_for_baseline = {}
for i in zip(file_name_list, input_sent_num_list):
  if (i[0] in dict_of_baselines.keys()):
    dict_of_selected_lines_for_baseline[i[0]] = []
    for j in i[1]: 
      if ((int)(j) in dict_of_baselines[i[0]]): #Main checking condition where the Ablation Analysis sentence is included only if it is a contribution sentence
        dict_of_selected_lines_for_baseline[i[0]] = dict_of_selected_lines_for_baseline[i[0]] + [(int)(j)]

In [ ]:
##### Phrases arrangement in ascending order according to their sentence numbers and their starting character numbers 
import copy
train_final_phrases_list = []
for i in range(len(input_stanza_list)):
  entity_list = [j.split('\t') for j in input_entity_list[i]]
  entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) 
  train_final_phrases_list = train_final_phrases_list + [entity_list]

In [ ]:
#### Here we form the triples using the rule that all three consequetive triples in the same sentence are treated as a triple.
baseline_predicted_triples = [] #Stores the triples formed for each file 

for j, name_of_file in enumerate(file_name_list):
  temp_store_list = []
  if name_of_file in dict_of_selected_lines_for_baseline.keys(): 
    for i in range(len(train_final_phrases_list[j])):
      if (int)(train_final_phrases_list[j][i][0]) in dict_of_selected_lines_for_baseline[name_of_file]: #Form triples only for those files in which "Ablation Analysis" sentences are found
        try:
          #Checks if the sentence number is same for the three consequetive phrases
          if((train_final_phrases_list[j][i][0]==train_final_phrases_list[j][i+1][0])  and (train_final_phrases_list[j][i][0]==train_final_phrases_list[j][i+2][0])):
            temp_store_list = temp_store_list + [[train_final_phrases_list[j][i][3], train_final_phrases_list[j][i+1][3], train_final_phrases_list[j][i+2][3]]]
        except Exception as e:
          continue
  baseline_predicted_triples = baseline_predicted_triples + [temp_store_list]

In [ ]:
#### Write the formed triples only in those files which have Ablation Analysis information unit already predicted in them
#### Note that we are not predicting Ablation Analysis Information unit.
#### We are only forming triples for the files which have Ablation Analysis Information Unit already predicted in them (from the Information unit prediction step).
for i in zip(file_name_list, baseline_predicted_triples):
  #Checks whether we have formed Ablation Analysis triples for this file AND whether this file already contains Ablation Analysis Information Unit.
  if i[0] in dict_of_selected_lines_for_baseline.keys() and ("ablation-analysis.txt" in os.listdir(input_dir+i[0]+"/"+"triples/")):
    output_file = open(input_dir + i[0] + "/" + "triples/" + "ablation-analysis.txt","w")
    output_file.write("(Contribution||has||Ablation analysis)" + "\n")
    for j in i[1]:
      output_file.write("(" + j[0] + "||" + j[1] + "||" + j[2]  + ")" + "\n")     
    output_file.close()
